# Migrate users from ArcGIS accounts to new Enterprise accounts

In [34]:

from arcgis.gis import *
gis = GIS("home") 
%autosave 60



Autosaving every 60 seconds


### Enter username being migrated

In [35]:
userstring = "peadar.oconnell"
orig_userid = userstring + "_RSPB"
new_userid =  userstring + "@rspb.org.uk_RSPB"

### Return old user account and custome role id

In [36]:
olduser = gis.users.get(orig_userid)
olduser


<User username:peadar.oconnell_RSPB>

### Create new enterprise account.

In [37]:
enterpriseuser = gis.users.get(new_userid)

if not enterpriseuser:

    nenewuser = gis.users.create(username = new_userid,                            
                            firstname = olduser.firstName,
                            lastname = olduser.lastName,
                            password = "pwdNotUsed",
                            email = olduser.email,
                            description = olduser.description,
                            role = "org_user",
                            provider = 'enterprise',
                           idp_username= olduser.email
                          ) 
else:
    print("Enterprise account already exists!")

Enterprise account already exists!


### Check new account has been created

In [38]:
enterpriseuser = gis.users.get(new_userid) #need to rerun this line for new users
enterpriseuser

<User username:Peadar.OConnell@rspb.org.uk_RSPB>

### Update new user to old users custom role. This needs to be done after new account is created with a default role

In [39]:
enterpriseuser.update_role(olduser.roleId)

True

### Share Groups to new account.###
This will change ownership of any groups owned and add user to other groups. This code fails for external groups such as Okrney Stoat so code ignores that group. Alos seems to have trouble with Hub groups.

In [40]:
usergroups = olduser['groups']

for group in usergroups :
    
    #print (group)
    
    #ignore Orkney Stoat group as owned by SNH and code fails, also fails on hub groups. neither of these types of group should be an issue for most users
    if group['id'] not in ['290fa9558d80471a8117c8f0398081ec','29e707d712404f478bcf06fccb440101','286d131ffaa7467b917ddb21ff6733aa','32e47d828cbc4877a390aa4e57942a1a']:
        grp = gis.groups.get(group['id'])
        if (grp.owner == orig_userid):
            #print(grp)
            grp.reassign_to(new_userid)
        else:
            print(grp)
            grp.add_users(new_userid)
            grp.remove_users([orig_userid]) #bug in api means you have to pass username in dictionary

<Group title:"enterprise migration" owner:adrian.hughes_RSPB>


### Copy over content and folder. Be careful with this one!###
This happens in 2 passes. First, reassign everything on the root folder of 'My Contents'. Then, loop over each folder, create the same folder in the new user account, and reassign items in each folder to the new user in the correct folder. Note the script throws up an error when it comes across items in the same folder that have the same name e.g layer1 (tile layer) and layer1 (tile package). This error can be ignored as item is still moved.
<br><font color=red> Does not move items that are shared to a group with update capability. Handled in following cell but this cell below must be run first as it creates the folders

In [42]:
usercontent = olduser.items()

folders = olduser.folders
for item in usercontent:
    print(item)
    try:        
        item.reassign_to(new_userid)        
    except Exception as e:
        print("Item may have been already assigned to the user.")

for folder in folders:
    print("creating folder..." + folder['title'])
    gis.content.create_folder(folder['title'], new_userid)
    folderitems = olduser.items(folder=folder['title'])
    
    for item in folderitems:
        print(item)
        try:
            item.reassign_to(new_userid, target_folder=folder['title'])
        except:
            ("likley duplicate error")

<Item title:"ArcGIS test map" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"Hotspot maps" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"Hotspots for ECC KI" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"Flamborough Hotspot" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"Skomer Hotspot" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"Rathlin hotspot" type:Web Map owner:peadar.oconnell_RSPB>
<Item title:"BLKI_UK_Eire_UsageMap_ExtraHiRes" type:Image owner:peadar.oconnell_RSPB>


### Content reassignment (update group method)
<font color=red>Does not work if user is not the owner of the update group. Possible bug in api</font>

In [43]:
usercontent = olduser.items()

for item in usercontent:
    print(item)
    groups = item.shared_with
    item.unshare(groups["groups"])
    item.reassign_to(new_userid)
    item.share(groups)    
    
folders = olduser.folders

for folder in folders:
    folderitems = olduser.items(folder=folder['title'])
    for folderitem in folderitems:
            print("folderitem" + str(folderitem))
            groups = item.shared_with
            item.unshare(groups["groups"])
            item.reassign_to(new_userid, target_folder=folder['title'])
            item.share(groups) 

### Need to handle content shared with update groups separately

### Assign ArcGIS Pro Licences ### 
View Pro licenses of old account


In [44]:
pro_license = gis.admin.license.get('ArcGIS Pro')
pro_licences = pro_license.user_entitlement(olduser.username)
pro_licences


{}

If old account has pro licence and extentions, assign  to new account and optionally by uncommenting line below to remove from old account

In [45]:
if pro_licences:
    pro_license.assign(username=enterpriseuser.username, entitlements=['desktopAdvN','3DAnalystN','dataInteropN','geostatAnalystN','imageAnalystN','networkAnalystN','spatialAnalystN'] , suppress_email=True)
    print ("licenses assigned")
    pro_license.revoke(username=olduser.username, entitlements='*')
    print("licenses revoed from old account")
else:
    print ("no licences assigned")
        

no licences assigned


### Assign credits ###

In [46]:
olduser.assignedCredits

0.0

In [47]:
#enterpriseuser.assignCredits = 300 #olduser.assignedCredits
gis.admin.credits.allocate(username=enterpriseuser.username, credits=olduser.assignedCredits)

True

In [48]:
enterpriseuser.assignedCredits # this number doesn't always reflect the update and seems to be affected by a time lag. Check AGOL to see update

-1.0

### Enable ESRI access for new account 

In [49]:
enterpriseuser.esri_access = 1

### disable old account###

In [50]:
olduser.disable()

True